In [5]:
import argparse
import configparser as cp
import logging
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
    
logger = logging.getLogger("inverse validation")

%cd ../..

# Now `current_directory` should reflect the new working directory
current_directory = os.getcwd()
print(current_directory)

savedir = current_directory + "src/inverse_validation/inverse_val_log_dir"
arch_cfg = "mm_test"
benchmark = "matmult.py"

# Add the parent directory to sys.path
sys.path.insert(0, current_directory)

# Now you can safely import modules that rely on the correct working directory
from src import coefficients
from src import symbolic_simulate
from src import hardwareModel
from src import hw_symbols
from src import optimize
from src import sim_util
from src import architecture_search

args = None

tech_nodes = [7, 5, 3]
initial_tech_params = {}
final_tech_params = {}
params_exclusion_list = []

/
/


In [6]:
def parse_output(f):
    # mostly copied from codesign.py
    tech_params = {}
    lines = f.readlines()
    mapping = {}
    i = 0
    while lines[i][0] != "x":
        i += 1
    while lines[i][0] == "x":
        mapping[lines[i][lines[i].find("[") + 1 : lines[i].find("]")]] = (
            hw_symbols.symbol_table[lines[i].split(" ")[-1][:-1]]
        )
        i += 1
    while i < len(lines) and lines[i].find("x") != 4:
        i += 1
    i += 2
    for _ in range(len(mapping)):
        key = lines[i].split(":")[0].lstrip().rstrip()
        value = float(lines[i].split(":")[2][1:-1])
        tech_params[mapping[key]] = value  # just know that self.tech_params contains all dat
        i += 1
    return tech_params

In [7]:
def plot_diff(tech_node_pair):
    # use final tech params to determine params to plot because not every tech param appears in edp equation
    tech_node_0_keys = final_tech_params[tech_node_pair[0]].keys()
    params_to_plot = [tech_param for tech_param in tech_node_0_keys if tech_param not in params_exclusion_list]

    tech_node_0_vals = [final_tech_params[tech_node_pair[0]][tech_param] for tech_param in params_to_plot]
    tech_node_1_vals = [initial_tech_params[tech_node_pair[1]][tech_param] for tech_param in params_to_plot]

    # ratio of optimized params of higher tech node to initial params of lower tech node
    ratios = [tech_node_0_vals[i] / tech_node_1_vals[i] for i in range(len(tech_node_1_vals))]
    i = 0
    while (i < len(params_to_plot)):
        num_params_on_fig = min(5, len(params_to_plot)-i)
        X_axis = np.arange(num_params_on_fig)
        plt.bar(X_axis, ratios[i:i+num_params_on_fig])
        plt.xticks(X_axis, params_to_plot[i:i+num_params_on_fig])
        plt.xlabel("tech params")
        plt.ylabel("tech param ratios")
        plt.title(f"Ratio of optimized tech params for {tech_node_pair[0]} nm and initial tech params for {tech_node_pair[1]} nm")
        plt.savefig(f"src/inverse_validation/figs/{tech_node_pair[0]}_{tech_node_pair[1]}_compare_{i/5}.png")
        plt.close()
        i += 5

In [8]:
def run_initial():
    edps = []
    hws = {}
    dfgs = {}
    for tech_node in tech_nodes:
        # initialize hw model, override architecture config to have current tech node
        logger.info(
            f"Setting up architecture search; benchmark: {benchmark}, config: {arch_cfg}"
        )
        (
            simulator,
            hw,
            computation_dfg,
        ) = architecture_search.setup_arch_search(benchmark, arch_cfg, True, tech_node)
        hw.init_memory(
            sim_util.find_nearest_power_2(simulator.memory_needed),
            sim_util.find_nearest_power_2(simulator.nvm_memory_needed),
        )
        scheduled_dfg = simulator.schedule(computation_dfg, hw)
        hardwareModel.un_allocate_all_in_use_elements(hw.netlist)
        simulator.simulate(scheduled_dfg, hw)
        simulator.calculate_edp()
        logger.warning(f"edp of simulation running on {tech_node} nm tech node: {simulator.edp} E-18 Js")
        
        rcs = hw.get_optimization_params_from_tech_params()
        initial_tech_params[tech_node] = sim_util.generate_init_params_from_rcs_as_symbols(rcs)
        
        edps.append(simulator.edp)
        hws[tech_node] = hw
        dfgs[tech_node] = scheduled_dfg
    return edps, hws, dfgs

In [9]:
def run_pairwise(tech_node_pair, improvement, hws, dfgs):
    logger.warning(f"tech node pair {tech_node_pair}, asking inverse pass for {improvement} edp improvement")
    symbolic_sim = symbolic_simulate.SymbolicSimulator()
    symbolic_sim.simulator_prep(benchmark, hws[tech_node_pair[0]].latency)
    coefficients.create_and_save_coefficients([hws[tech_node_pair[0]].transistor_size])
    hardwareModel.un_allocate_all_in_use_elements(hws[tech_node_pair[0]].netlist)

    symbolic_sim.simulate(dfgs[tech_node_pair[0]], hws[tech_node_pair[0]])
    symbolic_sim.calculate_edp(hws[tech_node_pair[0]])

    stdout = sys.stdout
    with open(f"{savedir}/ipopt_out_{tech_node_pair[0]}.txt", "w") as sys.stdout:
        optimize.optimize(initial_tech_params[tech_node_pair[0]], symbolic_sim.edp, "ipopt", improvement)
    sys.stdout = stdout
    f = open(f"{savedir}/ipopt_out_{tech_node_pair[0]}.txt", "r")
    final_tech_params[tech_node_pair[0]] = parse_output(f)
    logger.warning(f"final tech params for {tech_node_pair[0]} nm: {final_tech_params[tech_node_pair[0]]}")
    logger.warning(f"initial tech params for {tech_node_pair[1]} nm: {initial_tech_params[tech_node_pair[1]]}")

In [10]:
edps, hws, dfgs = run_initial()

AttributeError: 'NoneType' object has no attribute 'benchmark'

In [ ]:
for i in range(0, len(tech_nodes)-1):
    improvement = edps[i]/edps[i+1]
    run_pairwise(tech_nodes[i:i+2], improvement, hws, dfgs)

In [ ]:
for i in range(0, len(tech_nodes)-1):
    plot_diff(tech_nodes[i:i+2])